In [1]:
import pandas as pd
import re
import numpy as np
from time import time
import string
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import cross_val_score
#from nltk import FreqDist
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import gensim
from gensim import corpora
from gensim import models
from gensim.corpora.dictionary import Dictionary
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     D:\userdata\lianos\Application Data\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\lianos\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:843: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
en_stop = set(stopwords.words("english"))
digits = ['one','two','three','four','five','six','seven','eight','nine','ten','computer', 'will', 'develop', 'development',
                            'project', 'research', 'new', 'use', 
                            'europe', 'european','would','one','i']
for x in digits:
    en_stop.add(str(x))

In [3]:
list=["biology","cooking","crypto","diy","robotics","travel"]

Read each csv file individually and return dataframes cleaned up (no NA values, no symbols such as new paragraph or new line) 
Test dataframe is the Psysics subject

In [4]:
def get_df(x):
    path = 'stackexchange/{}.csv'.format(str(x))
    dataframe = pd.read_csv(path,usecols=['title','content','tags'],index_col=0)
    dataframe = dataframe.reset_index()
    dataframe.dropna()
    dataframe = dataframe.replace(re.compile('<.*?>'), ' ')
    dataframe = dataframe.replace(re.compile('\n'), ' ')
    return dataframe

biology = get_df('biology')
cooking = get_df('cooking')
crypto = get_df('crypto')
diy = get_df('diy')
robotics = get_df('robotics')
travel = get_df('travel')
test = get_df('test')   #physics set

Make a new dataframe called training_set consisting of the training set data. (all datasets concatenated)

In [5]:
training_list=[]
for x in list:
    path = 'stackexchange/{}.csv'.format(str(x))
    training_list.append(pd.read_csv(path,usecols=['title','content','tags'],index_col=0))
complete_set = pd.concat(training_list, axis=0 , join='outer')
complete_set.dropna()
complete_set= complete_set.reset_index()
complete_set.head(10)
complete_set = complete_set.replace(re.compile('<.*?>'), ' ')
complete_set = complete_set.replace(re.compile('\n'), ' ')

In [6]:
complete_set.shape

(87000, 3)

In [7]:
print(complete_set['tags'].values)

['ribosome binding-sites translation synthetic-biology' 'rna biochemistry'
 'immunology cell-biology hematology' ...,
 'customs-and-immigration officials registration macedonia' 'visas austria'
 'untagged']


Combine all columns of complete_set to a 3-tuple 

In [8]:
subset = complete_set[['title','content','tags']]
tuples = [tuple(x) for x in subset.values]
print(len(tuples))

87000


In [9]:
test.shape

(81926, 3)

In [10]:
test.head(20)

,id,title,content
0,What is spin as it relates to subatomic partic...,I often hear about subatomic particles having...,NaN
1,What is your simplest explanation of the strin...,How would you explain string theory to non ph...,NaN
2,"Lie theory, Representations and particle physics",This is a question that has been posted at ma...,NaN
3,Will Determinism be ever possible?,What are the main problems that we need to so...,NaN
4,Hamilton's Principle,Hamilton's principle states that a dynamic sy...,NaN
5,What is sound and how is it produced?,"I've been using the term ""sound"" all my life,...",NaN
6,What experiment would disprove string theory?,I know that there's big controversy between t...,NaN
7,Why does the sky change color? Why the sky is ...,Why does the sky change color? Why the sky is...,NaN
8,How's the energy of particle collisions calcul...,Physicists often refer to the energy of colli...,NaN
9,Monte Carlo use,Where is the Monte Carlo method used in physi...,NaN


In [11]:
print(' Cooking: ')
print(cooking.shape)
print(' Biology: ')
print(biology.shape)
print(' DIY: ')
print(diy.shape)
print(' Robotics: ')
print(robotics.shape)
print(' Travel: ')
print(travel.shape)
print(' Crypto: ')
print(crypto.shape)


 Cooking: 
(15404, 3)
 Biology: 
(13196, 3)
 DIY: 
(25918, 3)
 Robotics: 
(2771, 3)
 Travel: 
(19279, 3)
 Crypto: 
(10432, 3)


Get the titles from the complete set

In [12]:
titles_only=complete_set['title']
titles_only.shape

(87000,)

Let's get back to the test dataframe (consisting of physics-related stackexchange threads) and see if we can extract some common tags 

In [13]:
test.head(10)


,id,title,content
0,What is spin as it relates to subatomic partic...,I often hear about subatomic particles having...,NaN
1,What is your simplest explanation of the strin...,How would you explain string theory to non ph...,NaN
2,"Lie theory, Representations and particle physics",This is a question that has been posted at ma...,NaN
3,Will Determinism be ever possible?,What are the main problems that we need to so...,NaN
4,Hamilton's Principle,Hamilton's principle states that a dynamic sy...,NaN
5,What is sound and how is it produced?,"I've been using the term ""sound"" all my life,...",NaN
6,What experiment would disprove string theory?,I know that there's big controversy between t...,NaN
7,Why does the sky change color? Why the sky is ...,Why does the sky change color? Why the sky is...,NaN
8,How's the energy of particle collisions calcul...,Physicists often refer to the energy of colli...,NaN
9,Monte Carlo use,Where is the Monte Carlo method used in physi...,NaN


In [14]:
test['title'].shape

(81926,)

In [15]:
test_objectives= test['title'].str.lower().str.replace('|'.join([re.escape(x) for x in string.punctuation]), ' ')
test_objectives_split=test_objectives.str.split()
test_objectives_split.head(20)

0     [i, often, hear, about, subatomic, particles, ...
1     [how, would, you, explain, string, theory, to,...
2     [this, is, a, question, that, has, been, poste...
3     [what, are, the, main, problems, that, we, nee...
4     [hamilton, s, principle, states, that, a, dyna...
5     [i, ve, been, using, the, term, sound, all, my...
6     [i, know, that, there, s, big, controversy, be...
7     [why, does, the, sky, change, color, why, the,...
8     [physicists, often, refer, to, the, energy, of...
9     [where, is, the, monte, carlo, method, used, i...
10    [i, think, it, s, clear, enough, that, if, you...
11    [i, am, wondering, if, someone, could, provide...
12    [we, ve, learned, that, the, wave, function, o...
13    [i, recently, encountered, a, puzzle, where, a...
14    [what, is, einstein, s, theory, of, special, r...
15    [there, is, a, common, myth, that, water, flow...
16    [if, i, separate, two, magnets, whose, opposit...
17    [physicists, studying, the, grounds, of, p

In [16]:
#stopwords_current = set(en_stop) | additional_stopwords
test_objectives_split= test_objectives_split.apply(lambda tokens: [token for token in tokens if token not in en_stop])

Now we construct a dictionary from the above using gensim lib

In [17]:
objectives_dictionary = Dictionary(test_objectives_split)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(27610 unique tokens: ['perturbation', 'losing', 'molecular', 'bucks', 'thermopile']...)
INFO : adding document #20000 to Dictionary(37860 unique tokens: ['perturbation', 'losing', '082w', 'tumour', 'occours']...)
INFO : adding document #30000 to Dictionary(46292 unique tokens: ['squirt', 'molecular', 'essays', 'sentient', 'thrustratio']...)
INFO : adding document #40000 to Dictionary(53386 unique tokens: ['squirt', 'embedment', 'molecular', 'essays', 'sentient']...)
INFO : adding document #50000 to Dictionary(59437 unique tokens: ['squirt', 'embedment', 'molecular', 'essays', 'sentient']...)
INFO : adding document #60000 to Dictionary(65336 unique tokens: ['squirt', 'pseudovacuum', 'embedment', 'molecular', 'essays']...)
INFO : adding document #70000 to Dictionary(71351 unique tokens: ['squirt', 'surpassing', 'pseudovacuum', 'embedment', 'molecular']...)
INFO : adding document #8000

Now the corpus

In [18]:
class ObjectivesCorpus(object):
    def __init__(self, documents, dictionary):
        self.documents = documents
        self.dictionary = dictionary
    def __iter__(self):
        for document in self.documents:
            yield self.dictionary.doc2bow(document)
            
objective_corpus = ObjectivesCorpus(test_objectives_split,objectives_dictionary)

Run the LDA using gensim lib

In [19]:
t0 = time()
lda = gensim.models.ldamulticore.LdaMulticore(corpus=objective_corpus, 
                                              id2word=objectives_dictionary, 
                                              num_topics=10, 
                                              iterations=50,
                                              passes=5)
print("done in %0.3fs." % (time() - t0))

INFO : using symmetric alpha at 0.1
INFO : using symmetric eta at 1.2819198030971182e-05
INFO : using serial LDA version on this node
INFO : running online LDA training, 10 topics, 5 passes over the supplied corpus of 81926 documents, updating every 6000 documents, evaluating every ~60000 documents, iterating 50x with a convergence threshold of 0.001000
INFO : training LDA model using 3 processes
INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/81926, outstanding queue size 1
INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/81926, outstanding queue size 2
INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/81926, outstanding queue size 3
INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/81926, outstanding queue size 4
INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/81926, outstanding queue size 5
INFO : PROGRESS: pass 0, dispatched chunk #5 = documents up to #12000/81926, outstanding queue

done in 354.722s.


In [20]:
lda.print_topics(10)

INFO : topic #0 (0.100): 0.021*"theory" + 0.015*"field" + 0.010*"symmetry" + 0.009*"gauge" + 0.007*"string" + 0.007*"group" + 0.006*"fields" + 0.006*"model" + 0.005*"question" + 0.005*"3"
INFO : topic #1 (0.100): 0.021*"field" + 0.014*"water" + 0.014*"magnetic" + 0.014*"charge" + 0.012*"temperature" + 0.012*"electric" + 0.011*"pressure" + 0.010*"current" + 0.009*"energy" + 0.009*"heat"
INFO : topic #2 (0.100): 0.042*"1" + 0.036*"n" + 0.032*"psi" + 0.032*"rangle" + 0.027*"2" + 0.017*"0" + 0.016*"hat" + 0.015*"x" + 0.014*"h" + 0.014*"state"
INFO : topic #3 (0.100): 0.013*"energy" + 0.011*"electron" + 0.010*"light" + 0.007*"electrons" + 0.006*"frequency" + 0.005*"like" + 0.005*"question" + 0.005*"photon" + 0.005*"atom" + 0.005*"know"
INFO : topic #4 (0.100): 0.024*"force" + 0.015*"velocity" + 0.009*"mass" + 0.008*"object" + 0.008*"acceleration" + 0.008*"direction" + 0.007*"speed" + 0.006*"point" + 0.006*"ball" + 0.006*"axis"
INFO : topic #5 (0.100): 0.016*"light" + 0.014*"time" + 0.012*"e

[(0,
  '0.021*"theory" + 0.015*"field" + 0.010*"symmetry" + 0.009*"gauge" + 0.007*"string" + 0.007*"group" + 0.006*"fields" + 0.006*"model" + 0.005*"question" + 0.005*"3"'),
 (1,
  '0.021*"field" + 0.014*"water" + 0.014*"magnetic" + 0.014*"charge" + 0.012*"temperature" + 0.012*"electric" + 0.011*"pressure" + 0.010*"current" + 0.009*"energy" + 0.009*"heat"'),
 (2,
  '0.042*"1" + 0.036*"n" + 0.032*"psi" + 0.032*"rangle" + 0.027*"2" + 0.017*"0" + 0.016*"hat" + 0.015*"x" + 0.014*"h" + 0.014*"state"'),
 (3,
  '0.013*"energy" + 0.011*"electron" + 0.010*"light" + 0.007*"electrons" + 0.006*"frequency" + 0.005*"like" + 0.005*"question" + 0.005*"photon" + 0.005*"atom" + 0.005*"know"'),
 (4,
  '0.024*"force" + 0.015*"velocity" + 0.009*"mass" + 0.008*"object" + 0.008*"acceleration" + 0.008*"direction" + 0.007*"speed" + 0.006*"point" + 0.006*"ball" + 0.006*"axis"'),
 (5,
  '0.016*"light" + 0.014*"time" + 0.012*"earth" + 0.010*"black" + 0.009*"speed" + 0.009*"universe" + 0.009*"mass" + 0.009*"space"

LDA with TF-IDF

In [21]:
tf_idf = models.TfidfModel(objective_corpus)
tf_idf_corpus = tf_idf[objective_corpus]


INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : PROGRESS: processing document #10000
INFO : PROGRESS: processing document #20000
INFO : PROGRESS: processing document #30000
INFO : PROGRESS: processing document #40000
INFO : PROGRESS: processing document #50000
INFO : PROGRESS: processing document #60000
INFO : PROGRESS: processing document #70000
INFO : PROGRESS: processing document #80000
INFO : calculating IDF weights for 81926 documents and 78007 features (3787773 matrix non-zeros)


In [22]:
t0 = time()
lda_tf_idf = gensim.models.ldamulticore.LdaMulticore(corpus=objective_corpus, 
                                                     id2word=objectives_dictionary, 
                                                     num_topics=10, 
                                                     iterations=50,
                                                     passes=5)
print("done in %0.3fs." % (time() - t0))

INFO : using symmetric alpha at 0.1
INFO : using symmetric eta at 1.2819198030971182e-05
INFO : using serial LDA version on this node
INFO : running online LDA training, 10 topics, 5 passes over the supplied corpus of 81926 documents, updating every 6000 documents, evaluating every ~60000 documents, iterating 50x with a convergence threshold of 0.001000
INFO : training LDA model using 3 processes
INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/81926, outstanding queue size 1
INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/81926, outstanding queue size 2
INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/81926, outstanding queue size 3
INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/81926, outstanding queue size 4
INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/81926, outstanding queue size 5
INFO : PROGRESS: pass 0, dispatched chunk #5 = documents up to #12000/81926, outstanding queue

done in 367.619s.


In [23]:
lda_tf_idf.print_topics(20)

INFO : topic #0 (0.100): 0.062*"2" + 0.050*"r" + 0.039*"frac" + 0.039*"v" + 0.034*"1" + 0.028*"e" + 0.028*"0" + 0.023*"p" + 0.017*"n" + 0.014*"c"
INFO : topic #1 (0.100): 0.018*"quantum" + 0.013*"theory" + 0.013*"state" + 0.011*"particle" + 0.008*"field" + 0.008*"spin" + 0.008*"particles" + 0.007*"states" + 0.007*"question" + 0.007*"mechanics"
INFO : topic #2 (0.100): 0.017*"light" + 0.012*"earth" + 0.007*"could" + 0.007*"question" + 0.006*"black" + 0.006*"like" + 0.006*"speed" + 0.006*"universe" + 0.005*"see" + 0.005*"matter"
INFO : topic #3 (0.100): 0.009*"energy" + 0.007*"like" + 0.006*"know" + 0.005*"question" + 0.005*"find" + 0.005*"model" + 0.004*"using" + 0.004*"temperature" + 0.004*"number" + 0.004*"could"
INFO : topic #4 (0.100): 0.035*"energy" + 0.016*"field" + 0.016*"charge" + 0.012*"electron" + 0.011*"electrons" + 0.011*"electric" + 0.010*"temperature" + 0.009*"pressure" + 0.009*"potential" + 0.009*"heat"
INFO : topic #5 (0.100): 0.076*"2" + 0.060*"1" + 0.043*"0" + 0.032*"f

[(0,
  '0.062*"2" + 0.050*"r" + 0.039*"frac" + 0.039*"v" + 0.034*"1" + 0.028*"e" + 0.028*"0" + 0.023*"p" + 0.017*"n" + 0.014*"c"'),
 (1,
  '0.018*"quantum" + 0.013*"theory" + 0.013*"state" + 0.011*"particle" + 0.008*"field" + 0.008*"spin" + 0.008*"particles" + 0.007*"states" + 0.007*"question" + 0.007*"mechanics"'),
 (2,
  '0.017*"light" + 0.012*"earth" + 0.007*"could" + 0.007*"question" + 0.006*"black" + 0.006*"like" + 0.006*"speed" + 0.006*"universe" + 0.005*"see" + 0.005*"matter"'),
 (3,
  '0.009*"energy" + 0.007*"like" + 0.006*"know" + 0.005*"question" + 0.005*"find" + 0.005*"model" + 0.004*"using" + 0.004*"temperature" + 0.004*"number" + 0.004*"could"'),
 (4,
  '0.035*"energy" + 0.016*"field" + 0.016*"charge" + 0.012*"electron" + 0.011*"electrons" + 0.011*"electric" + 0.010*"temperature" + 0.009*"pressure" + 0.009*"potential" + 0.009*"heat"'),
 (5,
  '0.076*"2" + 0.060*"1" + 0.043*"0" + 0.032*"frac" + 0.032*"mu" + 0.024*"amp" + 0.017*"x" + 0.017*"3" + 0.017*"alpha" + 0.016*"nu"'),

Now time for some LDA using scikit learn

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 10


In [25]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(test['title'])
print("done in %0.3fs." % (time() - t0))

done in 7.431s.
